In [6]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
import pandas as pd
import itertools
import math
import os
import sympy as sp
import random
from sklearn.model_selection import train_test_split
from sympy import symbols, Function, diff
output_directory = r'../../../Data/3D_Lotka_Volterra/3D_Lotka_T5_N80'
def generate_random_values():
    x1 = random.uniform(0.05, 2)
    x2 = random.uniform(0.05, 2)
    x3 = random.uniform(0.05, 2)
    return x1, x2, x3
def calculate_c(x1, x2, x3):
    c = x1 + x2 + x3
    return c
def generate_random_values_based_on_c(c):
    min_val = 0.05
    max_val = 2
    x1 = random.uniform(min_val, min(max_val, c - min_val * 2))
    x2 = random.uniform(min_val, min(max_val, c - x1 - min_val))
    x3 = c - x1 - x2
    return x1, x2, x3
def generate_data(initial_conditions):
    def normalize(vector):
        norm = np.linalg.norm(vector)
        if norm == 0: 
            return vector
        return vector / norm
    def normalized_system(y, t):
        x1, x2, x3 = y
        f = np.array([x1 * (x2 - x3), x2 * (x3 - x1), x3 * (x1 - x2)])
        normalized_f = normalize(f)
        return normalized_f
    num_trajectories = 5
    t = np.linspace(0, 100, 80) # 10 data points per trajectory
    all_trajectory_data = []
    initial_conditions_to_print = []
    print("Initial data (x1, x2, x3):")
    for i, initial_condition in enumerate(initial_conditions):
        print(f"({initial_condition[0]}, {initial_condition[1]}, {initial_condition[2]})")
        sol = odeint(normalized_system, initial_condition, t)
        all_trajectory_data.append(sol)
    num_variables = 3 # Adjust number of variables that we need for the regression accordingly
    column_names = [f'x{i+1}' for i in range(num_variables)]
    column_names.append('trajectory')
    with open('50.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        for r, data in enumerate(all_trajectory_data):
            for j in range(len(t)):
                row = data[j].tolist() + [r + 1]
                writer.writerow(row) 
    output_directory1 = r'../../../results/3D_Lotka_Volterra/3D_Lotka_T5_N80'
    plt.figure(figsize=(10, 6))
    for i, sol in enumerate(all_trajectory_data):
        for j in range(sol.shape[1]):
            plt.plot(t, sol[:, j])
    plt.savefig(os.path.join(output_directory1, 'trajectory.png'))
    plt.close()
def split_data():
    trajectories = {}
    column_names = None
    with open('../../../Data/3D_Lotka_Volterra/3D_Lotka_T5_N80/trainingp_data50.csv', 'r') as trainfile:
        reader = csv.DictReader(trainfile)
        column_names = reader.fieldnames
        for row in reader:
            trajectory = float(row['trajectory'])
            if trajectory not in trajectories:
                trajectories[trajectory] = []
            trajectory_data = {key: float(value) for key, value in row.items()}
            trajectories[trajectory].append(trajectory_data)
    for traj_points in trajectories.values():
        random.shuffle(traj_points)
    num_points_per_file = len(next(iter(trajectories.values()))) // 5  # divide into five splits (n stratify)
    for i in range(5):  # Five-fold cross-validation
        output_filename = f'B50{i+1}.csv'
        with open(os.path.join(output_directory, output_filename), 'w', newline='') as output_file:
            writer = csv.DictWriter(output_file, fieldnames=column_names)
            writer.writeheader()
            for trajectory, points in trajectories.items():
                for point in points[i * num_points_per_file: (i + 1) * num_points_per_file]:
                    writer.writerow(point)
if __name__ == "__main__":
    x1, x2, x3 = generate_random_values()
    c = calculate_c(x1, x2, x3)
    initial_conditions = [generate_random_values_based_on_c(c) for _ in range(5)]
    generate_data(initial_conditions)
    data = np.genfromtxt('50.csv', delimiter=',', names=True)
    training_data = []
    holdout_data = []
    for r in range(1, 6):  # this represents the number of initial data is 5. i.e., (1,6) means 5 initial data
        trajectory_subset = data[data['trajectory'] == r]
        train_set, holdout_set = train_test_split(trajectory_subset, test_size=0.2, random_state=42)
        training_data.extend(train_set)
        holdout_data.extend(holdout_set)
    column_names = data.dtype.names
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'w', newline='') as trainfile:
        writer = csv.writer(trainfile)
        writer.writerow(column_names)
        for row in training_data:
            writer.writerow([row[col] for col in column_names])
    with open(os.path.join(output_directory, 'holdoutp_data50.csv'), 'w', newline='') as holdfile:
        writer = csv.writer(holdfile)
        writer.writerow(column_names)
        for row in holdout_data:
            writer.writerow([row[col] for col in column_names])
    split_data()

Initial data (x1, x2, x3):
(1.7867420999699821, 0.6598674101534446, 0.5690695765994008)
(1.2069010535467921, 0.41220400377877103, 1.3965740293972644)
(1.9207379682053776, 0.6325848081381542, 0.4623563103792958)
(0.8715649459518888, 0.5385413384098416, 1.6055728023610973)
(1.7478228763405523, 0.7403127956520152, 0.52754341473026)
